# Load dependencies

In [1]:
#load dependencies
from sql_conn import connection #SQL connection details and variables
#import neuralnet as nn #functions to form neural network and calculate accuracy and plot accuracy with history
import preproc as pr #functions to preprocess data 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

from sklearn import preprocessing #for one-hot encoding

For the Live SQL Server Authentication, please enter your


In [ ]:
#added comment for testing Git connection -- 20241114

# SQL Script

In [2]:
s='''
        SELECT DISTINCT
        ST.[StudyID] as [StudyID]
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,StudyHRCSHealthCategory

        FROM(
        SELECT 
        [StudyID]
        ,StudyHRCSHealthCategory
        from [CPMS_BI].[dbo].[ODP_StudyHRCS]
        WHERE StudyID is NOT NULL
		AND StudyHRCSHealthCategory <> 'Uncategorised') AS HR


        LEFT JOIN 
        (SELECT 
        StudyID
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,Study_IsCommercial
        ,Study_IsDraft

        FROM [CPMS_BI].[dbo].[ODP_Study]
        WHERE 1=1
        ) AS ST
        ON HR.StudyID = ST.StudyID
        WHERE 1=1

    '''

# with open("C:/Users/busaji/Documents/Projects/StudyClassifier-main/dat/sql-read.txt","r") as f:
#     s = f.read()

In [3]:
study_df = pd.read_sql(s, connection)

C:\Users\busaji\AppData\Local\Temp\ipykernel_14988\3074574563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  study_df = pd.read_sql(s, connection)


In [4]:
study_df.dtypes

StudyID                                        int64
StudyTitle                                    object
StudyShortName                                object
StudyResearchSummary                          object
StudyInclusionCriteria                        object
StudyDesignType                               object
StudyPhases_Concatenated                      object
StudySettings_Concatenated                    object
StudyManagingSpecialty                        object
StudyManagingSpecialty_PrimarySubSpecialty    object
StudyHRCSHealthCategory                       object
dtype: object

In [5]:
#pre-processing
study_df2 = study_df.fillna("not available")

#convert to float
#study_df['StudyResearchSummary'] = study_df['StudyResearchSummary'].astype("string")

In [6]:
study_df2

,StudyID,StudyTitle,StudyShortName,StudyResearchSummary,StudyInclusionCriteria,StudyDesignType,StudyPhases_Concatenated,StudySettings_Concatenated,StudyManagingSpecialty,StudyManagingSpecialty_PrimarySubSpecialty,StudyHRCSHealthCategory
0,646,A Randomised Trial of High Dose Therapy in Loc...,RT01,To compare standard 64 Gy dose of conformal ra...,1) Histologically confirmed cancer of the pros...,Interventional,III,Primary Care,Cancer,Prostate Cancer,Cancer and neoplasms
1,651,A Randomised trial of Hormone Therapy plus Rad...,PR07,To evaluate any possible benefit from the addi...,Histologically confirmed Prostate cancer withi...,Interventional,III,Primary Care,Cancer,Prostate Cancer,Cancer and neoplasms
2,760,The ProtecT Trial - Evaluating the effectivene...,ProtecT,The overall aim is to evaluate the effectivene...,- inclusion for randomisation: men with locali...,Interventional,III,"Primary Care, Secondary Care",Cancer,Prostate Cancer,Cancer and neoplasms
3,791,INTERmittent or CONTINuos ENdocrine Therapy Af...,Intercontinental,A phase III randomised trial comparing intermi...,Patients with adenocarcinoma of the prostate w...,Interventional,III,Primary Care,Cancer,Prostate Cancer,Cancer and neoplasms
4,798,Intermittent androgen deprivation in patients ...,EORTC 30985,not available,not specified,Interventional,III,"Secondary Care, Tertiary Care",Cancer,Prostate Cancer,Cancer and neoplasms
...,...,...,...,...,...,...,...,...,...,...,...
27711,65230,Treatment of severe Diabetic macular oedema wi...,The DAME Study,The PPI was involved from the very first steps...,Adults (>18 years) with diabetes type 1 or typ...,Interventional,N/A,Secondary Care,Ophthalmology,Anterior Segment Cataract,Renal and Urogenital
27712,65301,Supersaturated oxygen therapy using radial art...,Heart attack blood oxygen therapy study,"'Heart attack', known as acute ST-segment elev...",1. Age ≥18 years. 2. Ischaemic time ≤6 hours ...,Interventional,N/A,Tertiary Care,Cardiovascular,Atherothrombosis,Cardiovascular
27713,65360,Improving testing for cardiometabolic diseases...,Randomised trial of SMS to improve testing for...,The way we usually test new ways of supporting...,• Diagnosis of gestational diabetes • 18 to ...,Interventional,N/A,Primary Care,"Diabetes, Metabolic and Endocrine",Diabetes - Other,Metabolic and Endocrine
27714,65397,A pilot randomised controlled trial of an inte...,Reducing children’s exposure to second-hand sm...,The harmful health effects of children’s expos...,1. Parents and carers who are at least 18 year...,Observational,N/A,Community,Public Health,Public Health,Generic health relevance


In [6]:
#get study ID & HRCS tag
study_hrcs = study_df2[["StudyID","StudyHRCSHealthCategory"]]

In [7]:
#creating another subset of dataset without healthcategory
study_only = study_df2.drop("StudyHRCSHealthCategory", axis=1).drop_duplicates(ignore_index=True)


# One-Hot encoding for HRCS tags

In [8]:
#load onehotencoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', 
                    categories='auto', 
                    drop='if_binary')

enc.fit(study_hrcs)


OneHotEncoder(drop='if_binary', handle_unknown='ignore')

In [9]:
study_hrcs
study_only

,StudyID,StudyHRCSHealthCategory
0,646,Cancer and neoplasms
1,651,Cancer and neoplasms
2,760,Cancer and neoplasms
3,791,Cancer and neoplasms
4,798,Cancer and neoplasms
...,...,...
27711,65230,Renal and Urogenital
27712,65301,Cardiovascular
27713,65360,Metabolic and Endocrine
27714,65397,Generic health relevance


In [9]:
#pandas one-hot encoding
df_encoded = pd.concat([study_hrcs, pd.get_dummies(study_hrcs['StudyHRCSHealthCategory'])], axis=1).groupby(['StudyID']).sum().reset_index()
# df_encoded = pd.get_dummies(study_hrcs, dtype=int, columns=['StudyHRCSHealthCategory'])

In [12]:
# import numpy as np
# df_encoded=np.unique(df_encoded, axis=0)

In [10]:
df_encoded.shape
#should be 23 shapes - 22 ICD Chapters and 1 blank

(24164, 23)

In [11]:
df_encoded.drop("StudyHRCSHealthCategory", axis=1, inplace=True)

In [12]:
#Merge source data and df_encoded together

data = pd.merge(
    left=study_only,
    right=df_encoded,
    how='right',
    on='StudyID'
)

In [13]:
data.shape

(24164, 31)

In [14]:
data

,StudyID,StudyTitle,StudyShortName,StudyResearchSummary,StudyInclusionCriteria,StudyDesignType,StudyPhases_Concatenated,StudySettings_Concatenated,StudyManagingSpecialty,StudyManagingSpecialty_PrimarySubSpecialty,...,Mental health,Metabolic and Endocrine,Musculoskeletal,Neurological,Oral and Gastrointestinal,Renal and Urogenital,Reproductive health and childbirth,Respiratory,Skin,Stroke
0,646,A Randomised Trial of High Dose Therapy in Loc...,RT01,To compare standard 64 Gy dose of conformal ra...,1) Histologically confirmed cancer of the pros...,Interventional,III,Primary Care,Cancer,Prostate Cancer,...,0,0,0,0,0,0,0,0,0,0
1,651,A Randomised trial of Hormone Therapy plus Rad...,PR07,To evaluate any possible benefit from the addi...,Histologically confirmed Prostate cancer withi...,Interventional,III,Primary Care,Cancer,Prostate Cancer,...,0,0,0,0,0,0,0,0,0,0
2,760,The ProtecT Trial - Evaluating the effectivene...,ProtecT,The overall aim is to evaluate the effectivene...,- inclusion for randomisation: men with locali...,Interventional,III,"Primary Care, Secondary Care",Cancer,Prostate Cancer,...,0,0,0,0,0,0,0,0,0,0
3,791,INTERmittent or CONTINuos ENdocrine Therapy Af...,Intercontinental,A phase III randomised trial comparing intermi...,Patients with adenocarcinoma of the prostate w...,Interventional,III,Primary Care,Cancer,Prostate Cancer,...,0,0,0,0,0,0,0,0,0,0
4,798,Intermittent androgen deprivation in patients ...,EORTC 30985,not available,not specified,Interventional,III,"Secondary Care, Tertiary Care",Cancer,Prostate Cancer,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24159,66617,Left bundle pacing monitoring study,Left bundle pacing monitoring study,Cohort of 20 patients with left bundle pacemak...,Patients with left bundle branch pacemakers,Observational,N/A,Hospitals,Cardiovascular,Cardiovascular,...,0,0,0,0,0,0,0,0,0,0
24160,66635,Blood RAman Spectroscopy In Lung cancer: towar...,BRASIL,Title: Blood RAman Spectroscopy In Lung cancer...,• Age 18 years or over • Confirmed histopatho...,Observational,II,"Hospitals, Secondary Care",not available,not available,...,0,0,0,0,0,0,0,1,0,0
24161,66642,Digital Decision Support to Reduce Time to Tre...,Digital Trauma Handover,Trauma causes approximately one out of ten dea...,This study involves two key populations: the p...,Observational,N/A,Tertiary Care,Trauma and Emergency Care,Emergency Medicine,...,0,0,0,0,0,0,0,0,0,0
24162,66657,Development of non-invasive biomarkers to dire...,Development of non-invasive biomarkers to dire...,Wheezing in preschool children (aged 1-5 years...,"Aim 1,2 and 3: Children aged 1-5 years EIT...",Both,N/A,Hospitals,Children,Respiratory and Cystic Fibrosis,...,0,0,0,0,0,0,0,1,0,0


# Pre-processing

In [15]:
#combine text data into one column
data['Text'] = data['StudyTitle'] + " " + data['StudyResearchSummary'] + " " + data['StudyInclusionCriteria']

In [16]:
#Setting StudyID as dataframe index
data.set_index('StudyID', inplace=True)

In [ ]:
data

In [17]:
#drop unrequired fields

data_cln = data.drop(['StudyTitle','StudyResearchSummary','StudyInclusionCriteria',
            'StudyTitle',
          'StudyDesignType','StudyPhases_Concatenated','StudySettings_Concatenated',
          'StudyShortName',
          'StudyManagingSpecialty','StudyManagingSpecialty_PrimarySubSpecialty'], axis=1)

In [ ]:
data_cln

In [18]:
#split dependent and independent features
X = data_cln['Text'] #independentfeature
y=data_cln.drop('Text', axis=1) # ICD-10 chapter columns

# TF-IDF Vectorizer

In [19]:
#test code for tf-idf and then multi-class multi-label classifier

vectorizer = TfidfVectorizer(max_features=5000, max_df=0.7)
vectorizer.fit(X)

TfidfVectorizer(max_df=0.7, max_features=5000)

In [20]:
#split in train and test
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
  
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [11]:
#medium embeddings model for version 0.5.4 -- NOTE that we have istalled v0.5.5 for scispacy 
%pip install scispacy spacy
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_md-0.5.4.tar.gz


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/119.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/119.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/119.1 MB ? eta -:--:--
     -------------------------------------- 0.1/119.1 MB 327.7 kB/s eta 0:06:04
     -------------------------------------- 0.1/119.1 MB 409.6 kB/s eta 0:04:51
     -------------------------------------- 0.1/119.1 MB 409.6 kB/s eta 0:04:51
     -------------------------------------- 0.2/119.1 MB 577.4 kB/s eta 0:03:27
     -------------------------------------- 0.2/119.1 MB 577.4 kB/s eta 0:03:27
     -------------------------------------- 0.2/119.1 MB 577.4 kB/s eta 0:03:27
     -------------------------------------- 0.3/119.1 MB 605.3 kB/s eta 0:03:17
     -------------------------------------- 0.4/119.1 MB 825.0 kB/s eta 0:02:24
     -------------------------------------- 0.4/119.1 MB 825.0 kB/s eta 0:02:24
     -------------------------------------- 0.5/119.1 MB 944.


[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Scispacy


In [12]:
import numpy as np
import spacy
# import en_core_sci_md

nlp = spacy.load("en_core_sci_md")  # Use "en_core_sci_md

def vectorize_text(texts):
    return np.array([nlp(text).vector for text in texts])


doc = nlp("Alterations in the hypocretin receptor 2 and preprohypocretin genes produce narcolepsy in some animals.")
# Convert text to vectors
# X = vectorize_text(texts)
# y = np.array(labels)
doc

c:\Users\busaji\Documents\Projects\StudyClassifier-main\.venv\Lib\site-packages\spacy\language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


Alterations in the hypocretin receptor 2 and preprohypocretin genes produce narcolepsy in some animals.

In [14]:
# Get document vector (average of token vectors)
doc_vector = doc.vector

print(doc_vector.shape)  # Output: (96,) for sm model, (200,) for md, (600,) for lg
for token in doc:
    print(f"Token: {token.text}, Vector: {token.vector[:5]}")  # Showing first 5 dimensions


(200,)
Token: Alterations, Vector: [-0.254577  -0.162247   0.142528   0.155708   0.0253562]
Token: in, Vector: [ 0.190115  -0.0282515  0.271463  -0.0823982  0.0915936]
Token: the, Vector: [ 0.0538258 -0.176276   0.0993653 -0.0491333  0.0211865]
Token: hypocretin, Vector: [-0.887773  0.63847   0.142412  0.165837  0.131899]
Token: receptor, Vector: [-0.274829   0.0885202  0.115995  -0.0621077  0.315455 ]
Token: 2, Vector: [ 0.208598  -0.318573   0.120229   0.0302139  0.109442 ]
Token: and, Vector: [ 0.115095   -0.0811072   0.0899549  -0.00734195 -0.00132037]
Token: preprohypocretin, Vector: [-0.55555    0.567101   0.0969877  0.160323  -0.0169081]
Token: genes, Vector: [-0.283578    0.00316874  0.216725   -0.153178   -0.0313043 ]
Token: produce, Vector: [-0.0744016  -0.00729794  0.067956   -0.0906845   0.237049  ]
Token: narcolepsy, Vector: [-0.765795   0.21212   -0.0984301 -0.151586   0.0517728]
Token: in, Vector: [ 0.190115  -0.0282515  0.271463  -0.0823982  0.0915936]
Token: some, Vect

2. Pre-processing for Spacy implementation

In [ ]:

def spacy_preproc(text):

    df["tokens"] = df["text_column"].apply(
        lambda x: [token.text for token in nlp(str(x))]) 
    #tokenization
    df["lemmas"] = df["text_column"].apply(
        lambda x: [token.lemma_ for token in nlp(str(x))]) #lemmatization - root form of words
    df["entities"] = df["text_column"].apply(
        lambda x: [(ent.text, ent.label_) for ent in nlp(str(x)).ents]) #- NER - Named Entitiy Recognition
    df["doc_vector"] = df["text_column"].apply(
        lambda x: nlp(str(x)).vector) #applying vectorization to the entire document


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Convert document vectors to NumPy array
X = np.vstack(df["doc_vector"].values)
y = df["label_column"].values  # Replace with the actual label column

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Multi-class Multi-Label Classfication

In [21]:
#multi-classifier model and prediction
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression


clf = MultiOutputClassifier(LogisticRegression()).fit(X_train_tfidf, y_train)

In [22]:
#prediction
prediction = clf.predict(X_test_tfidf)
prediction

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [23]:
#evaluation
from sklearn.metrics import accuracy_score
print('Accuracy Score: ', accuracy_score(y_test, prediction))

Accuracy Score:  0.6033519553072626


In [43]:
#random forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100, random_state = 1 )
multi_forest = MultiOutputClassifier(forest).fit(X_train_tfidf, y_train)
## Make prediction
dfOutput_multi_forest = multi_forest.predict_proba(X_test_tfidf)

In [44]:
#store prediction and check accuracy

forest_prediction = multi_forest.predict(X_test_tfidf)

In [45]:
print('Accuracy Score: ', accuracy_score(y_test, forest_prediction))

Accuracy Score:  0.6049979000419992


In [28]:
# feature importance from random forest:
import numpy as np
feat_impts = [] 
for clf in multi_forest.estimators_:
    feat_impts.append(clf.feature_importances_)

np.mean(feat_impts, axis=0)

array([2.43989858e-04, 5.72654844e-05, 5.51617222e-04, ...,
       6.26236492e-05, 6.95234487e-05, 3.28897310e-05])

# Grid-Search
1. This is for selecting/evalutating different models

In [26]:
# Sourced from ChatGPT 3.5 ----20250123
# Define classifiers and their hyperparameters
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.datasets import load_iris

grid_params = {
    # 'RandomForest': {
    #     'model': RandomForestClassifier(),
    #     'params': {
    #         'n_estimators': [10, 50, 100],
    #         'max_depth': [None, 10, 20],
    #         'min_samples_split': [2, 5, 10]
    #     }
    # },
    # 'SVM': {
    #     'model': SVC(),
    #     'params': {
    #         'C': [0.1, 1, 10],
    #         'kernel': ['linear', 'rbf'],
    #         #'gamma': ['scale', 'auto']
    #         'gamma': [0.1, 1, 10],
    #     }
    # },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.1, 1, 10],
            'solver': ['lbfgs', 'liblinear']
        }
    }
}

# Perform grid search for each classifier
best_models = {}
for name, clf in grid_params.items():
    print(f"Performing Grid Search for {name}...")
    grid_search = GridSearchCV(clf['model'], clf['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_tfidf, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Best parameters for {name}: {grid_search.best_params_}")
    print(f"Best CV accuracy for {name}: {grid_search.best_score_}")
    print("\n")

# Evaluate best models on the test set
for name, model in best_models.items():
    print(f"Evaluating {name} on test data...")
    y_pred = model.predict(X_test_tfidf)
    print(classification_report(y_test, y_pred))
    print("\n")


Performing Grid Search for KNN...
Best parameters for KNN: {'n_neighbors': 3, 'weights': 'distance'}
Best CV accuracy for KNN: 0.6498886814642426


Performing Grid Search for LogisticRegression...


ValueError: 
All the 18 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\busaji\Documents\Projects\StudyClassifier-main\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\busaji\Documents\Projects\StudyClassifier-main\.venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\busaji\Documents\Projects\StudyClassifier-main\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1201, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File

In [ ]:
''''
'gamma': [0.1, 1, 10],
param_grid = {'C': [0.1, 1,10],
              'gamma': [0.1, 1, 10],
              'kernel': ['rbf','linear']}

# Create a GridSearchCV object
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)

'''''

# Function for prediction (TEST) below:

In [ ]:
def multi_label_classification(X_train, y_train, X_test):
    """
    This function builds and evaluates a multi-label classification model using Scikit-Learn.
    It trains a Logistic Regression model, and prints the classification report.
    """
    # Initialize the base classifier
    base_classifier = LogisticRegression(max_iter=1000)
    # Wrap the base classifier with MultiOutputClassifier
    multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs=-1)
    # Train the multi-label classifier
    multi_target_classifier.fit(X_train, y_train)
    # Predict the labels for the test set
    y_pred = multi_target_classifier.predict(X_test)
    # Evaluate the model
    report = classification_report(y_test, y_pred, target_names=[f'Label {i}' for i in range(y.shape[1])])
    # Print the classification report
    print(report)

In [ ]:
def multi_label_classification(X_train, y_train, X_test, y_test):
       # Convert text data to TF-IDF features
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    # Initialize the base classifier
    base_classifier = LogisticRegression(max_iter=1000)
    # Wrap the base classifier with MultiOutputClassifier
    multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs=-1)
    # Train the multi-label classifier
    multi_target_classifier.fit(X_train_tfidf, y_train)
    # Predict the labels for the test set
    y_pred = multi_target_classifier.predict(X_test_tfidf)
    # Evaluate the model
    report = classification_report(y_test, y_pred, target_names=[f'Label {i}' for i in range(len(y_train[0]))])
    # Print the classification report
    print(report)